In [1]:
from google.colab import drive, output
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install wandb --upgrade
!wandb login

import wandb

output.clear()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 14.7 MB/s 
     |████████████████████████████████| 181 kB 47.9 MB/s 
     |████████████████████████████████| 147 kB 45.5 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=fb00bade9889698560801ec7912b25a60232c0ffd8f074a5d721655d343c53b1
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

In [ ]:
cd '/content/drive/MyDrive/UdeA/Codes/ECOVAE/src/'

In [1]:
!python train.py   \
    --folder_path '/content/drive/MyDrive/Datasets/Grabaciones_Camilo_2/Sitio1/AudioMoth' \
                  '/content/drive/MyDrive/Datasets/Grabaciones_Camilo_2/Sitio2/AudioMoth' \
                  '/content/drive/MyDrive/UdeA/Datasets/Grabaciones_Camilo_2/Sitio1/a/Data' \
                  '/content/drive/MyDrive/UdeA/Datasets/Grabaciones_Camilo_2/Sitio1/b/Data' \
                  '/content/drive/MyDrive/UdeA/Datasets/GrabacionesCamilo/G21A' \
                  '/content/drive/MyDrive/UdeA/Datasets/GrabacionesCamilo/G21S' \
    --audio_len 60 \
    --lwin 12 \
    --ext 'WAV' \
    --n_fft 1028 \
    --checkpoints_path '/content/drive/MyDrive/UdeA/Codes/ECOVAE/checkpoints' \
    --batch-size 16 \
    --num_epochs 10 \
    --num_hiddens 64 \
    --embedding_dim 512 \
    --num_embeddings 256 \
    --commitment_cost 0.25 \
    --decay 0.99 \
    --learning_rate 1e-2 \
    --wandb True \


python3: can't open file 'train.py': [Errno 2] No such file or directory


In [ ]:
!python train.py   \
    --folder_path '/content/drive/MyDrive/Datasets/Grabaciones_Camilo_2/Sitio1/AudioMoth' \
                  '/content/drive/MyDrive/Datasets/Grabaciones_Camilo_2/Sitio2/AudioMoth' \
                  '/content/drive/MyDrive/UdeA/Datasets/Grabaciones_Camilo_2/Sitio1/a/Data' \
                  '/content/drive/MyDrive/UdeA/Datasets/Grabaciones_Camilo_2/Sitio1/b/Data' \
                  '/content/drive/MyDrive/UdeA/Datasets/GrabacionesCamilo/G21A' \
                  '/content/drive/MyDrive/UdeA/Datasets/GrabacionesCamilo/G21S' \
    --audio_len 60 \
    --lwin 12 \
    --ext 'WAV' \
    --n_fft 1028 \
    --checkpoints_path '/content/drive/MyDrive/UdeA/Codes/ECOVAE/checkpoints' \
    --batch-size 32 \
    --num_epochs 10 \
    --num_hiddens 64 \
    --embedding_dim 512 \
    --num_embeddings 64 \
    --commitment_cost 0.25 \
    --decay 0.99 \
    --learning_rate 1e-3\
    --wandb True \
    --num_residual_layers 32 \
    --num_residual_hiddens 2\

In [100]:
in_channels = 1
num_hiddens = 128
embedding_dim = 512
num_residual_layers  = 32
num_residual_hiddens = 2


import torch
import torch.nn as nn

import torchaudio.transforms as T


sr = 22050
t = 12


xw = torch.randn((1,int(sr*t)))
data = T.Spectrogram(      
    n_fft=1028,
    win_length=None,
    hop_length=None,
    center=True,
    pad_mode="reflect",
    power=2.0)(xw)

print(f'Spectrogram: {data.shape}')


conv_1 = nn.Conv2d(in_channels=in_channels,
                          out_channels=num_hiddens // 2,
                          kernel_size=(5, 5),
                          stride=(2, 2), padding=(0, 0))(data)

conv_2 = nn.Conv2d(in_channels=num_hiddens // 2,
                          out_channels=num_hiddens,
                          kernel_size=(4, 4),
                          stride=(2, 2), padding=(0, 0))(conv_1)


residual_stack = ResidualStack(in_channels=num_hiddens,
                          num_hiddens=num_hiddens,
                          num_residual_layers=num_residual_layers,
                          num_residual_hiddens=num_residual_hiddens)(conv_2)





pre_vq_conv = nn.Conv2d(in_channels=num_hiddens,
                          out_channels=embedding_dim,
                          kernel_size=(3, 3),
                          stride=(2, 2))(residual_stack)


print(f'c1: {conv_1.shape} \t c2: {conv_2.shape} \t  residual: {residual_stack.shape} \t  vqconv: {pre_vq_conv.shape}')

residual_stack_d = ResidualStack(in_channels=embedding_dim,
                                        num_hiddens=embedding_dim,
                                        num_residual_layers=num_residual_layers,
                                        num_residual_hiddens=num_residual_hiddens)(pre_vq_conv)

conv_trans_1 = nn.ConvTranspose2d(in_channels=embedding_dim,
                                        out_channels=num_hiddens,
                                        kernel_size=(3, 3),
                                        stride=(2, 2), output_padding = (0,0))(residual_stack_d)


conv_trans_2 = nn.ConvTranspose2d(in_channels=num_hiddens,
                                        out_channels=num_hiddens // 2,
                                        kernel_size=(4, 4),
                                        stride=(2, 2), output_padding = (0,0))(conv_trans_1)

conv_trans_3 = nn.ConvTranspose2d(in_channels=num_hiddens // 2,
                                        out_channels=1,
                                        kernel_size=(5, 5),
                                        stride=(2, 2))(conv_trans_2)

print(f'ct1: {residual_stack_d.shape} \t ct1: {conv_trans_1.shape} \t ct2: {conv_trans_2.shape} \t ct3: {conv_trans_3.shape} ')

Spectrogram: torch.Size([1, 515, 515])
c1: torch.Size([64, 256, 256]) 	 c2: torch.Size([128, 127, 127]) 	  residual: torch.Size([128, 127, 127]) 	  vqconv: torch.Size([512, 63, 63])
ct1: torch.Size([512, 63, 63]) 	 ct1: torch.Size([128, 127, 127]) 	 ct2: torch.Size([64, 256, 256]) 	 ct3: torch.Size([1, 515, 515]) 


In [ ]:
audio_len = sr*12
resize = audio_len - xw.shape[1]

xr = torch.cat((xw,torch.zeros((1,resize)) ), dim=1) 


In [ ]:
torch.zeros((1,resize)).shape

torch.Size([1, 11025])

In [ ]:
xw.shape

torch.Size([1, 253575])